In [ ]:
%load_ext autoreload
%autoreload 2
import pygsti
import copy
from pygsti import tools as pgt
from pygsti import modelmembers as pgmm
from pygsti.protocols.gst import ModelEstimateResults
from pygsti.modelpacks import smq1Q_XY, smq1Q_ZN
from typing import Union, Dict
from pygsti.tools.leakage import add_lago_models, leaky_qubit_model_from_pspec, leakage_friendly_basis_2plus1
from pygsti.data import simulate_data
from pygsti.protocols import StandardGST, ProtocolData
import numpy as np

In [ ]:
mp = smq1Q_XY
ed = mp.create_gst_experiment_design(max_max_length=32)
lfb = leakage_friendly_basis_2plus1()
tm3 = leaky_qubit_model_from_pspec(mp.processor_spec(), basis=lfb)
ds = simulate_data(tm3, ed.all_circuits_needing_data, num_samples=1000, seed=1997)
gst = StandardGST( modes=('CPTPLND',), target_model=tm3, verbosity=2)
pd = ProtocolData(ed, ds)
res = gst.run(pd)

In [ ]:
def write_leakage_friendly_html_report(
        report_title: str,
        report_dir  : str,
        results : ModelEstimateResults,
        est_key: str ='CPTPLND',
        gaugeopt_verbosity : int = 0
    ):
    from copy import deepcopy
    results = deepcopy(results)
    add_lago_models(results, est_key, verbosity=gaugeopt_verbosity)
    from pygsti.report import construct_standard_report
    report = construct_standard_report(results, title=report_title, advanced_options={'n_leak': 1})
    report.write_html(report_dir)
    return

In [ ]:
report_dir = ''  # set me!

In [ ]:
write_leakage_friendly_html_report('leakage report', report_dir, res)